In [ ]:
from utils import load_api_key
API_KEY = load_api_key()

In [ ]:
# Test if the database is accessible

from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../../data/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

# Build agent

In [ ]:
#from langchain_ollama import ChatOllama
#llm = ChatOllama(model="llama3.2")

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", api_key=API_KEY)

In [ ]:
# from langchain import hub

## Source: https://smith.langchain.com/hub/langchain-ai/sql-query-system-prompt
# query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")
# query_prompt_template.messages[0].pretty_print()

In [ ]:
prompt = """Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}"""

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools=tools, prompt=prompt, debug=False)

In [ ]:
events = agent_executor.stream(
    {"messages": [("user", "What are the names of the first 10 artists in the database which starts with the letter P?")]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
events = agent_executor.stream(
    {"messages": [("user", "¿Qué país es el que más gasta?")]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
events = agent_executor.stream(
    {"messages": [("user", "¿Cuantas ventas ha habido en Londres?")]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
events = agent_executor.stream(
    {"messages": [("user", "Dime todos los artistas con alguna venta en Oslo. Límitalo a 20 resultados. Dame el resultado en una lista con emojis. Don't forget to join the TrackId")]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()